<a href="https://colab.research.google.com/github/armandossrecife/my_validation/blob/main/evaluate_inspection_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import dataset

In [1]:
!rm -rf *.xlsx

In [2]:
!wget https://github.com/Technical-Debt-Large-Scale/my_validation/raw/main/cassandra_issues_inspected_merged.xlsx

--2023-12-22 13:07:17--  https://github.com/Technical-Debt-Large-Scale/my_validation/raw/main/cassandra_issues_inspected_merged.xlsx
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Technical-Debt-Large-Scale/my_validation/main/cassandra_issues_inspected_merged.xlsx [following]
--2023-12-22 13:07:18--  https://raw.githubusercontent.com/Technical-Debt-Large-Scale/my_validation/main/cassandra_issues_inspected_merged.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 573631 (560K) [application/octet-stream]
Saving to: ‘cassandra_issues_inspected_merged.xlsx’

cassandra_issues_in 100%[=============

# Support functions


In [3]:
import pandas as pd
import collections
import matplotlib.pyplot as plt
import seaborn as sns
import random
import scipy.stats as stats
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

def convert_to_upper(list_of_strings):
  upper_list = []
  for string in list_of_strings:
    upper_list.append(string.upper())
  return upper_list

def conta_yes_no(my_list):
  conta_yes, conta_no = 0,0
  for each in my_list:
    if each == 'YES':
      conta_yes += 1
    else:
      conta_no += 1
  return conta_yes, conta_no

def convert_yes_no_to_int(dfa, dfb, my_column):
  labels = ["YES", "NO"]  # Define your possible labels here

  # Convert labels to numerical values
  label_to_num = {label: i for i, label in enumerate(labels)}
  dfa[my_column] = dfa[my_column].map(label_to_num)
  dfb[my_column] = dfb[my_column].map(label_to_num)
  return dfa, dfb

# Define a function to calculate Cohen's Kappa
def calculate_cohens_kappa(dfa, dfb, my_column):
    # Calculate Cohen's Kappa
    kappa = cohen_kappa_score(dfa[my_column], dfb[my_column])
    return kappa

def extra_metrics_yes(dfa, dfb, my_column):
  # Define your ground truth (actual values) and predicted values
  ground_truth = dfa[my_column]  # Ground truth values from manual inspection
  predicted = dfb[my_column]     # Predicted values from ChatGPT inspection

  # Calculate precision, recall, accuracy, and F1-score
  precision = precision_score(ground_truth, predicted, pos_label=1)
  recall = recall_score(ground_truth, predicted, pos_label=1)
  accuracy = accuracy_score(ground_truth, predicted)
  f1 = f1_score(ground_truth, predicted, pos_label=1)
  return precision, recall, accuracy, f1

# Dataset

Merge da inspeção manual e inspeção via ChatGPt

In [4]:
df_inspection = pd.read_excel('cassandra_issues_inspected_merged.xlsx')
df_inspection

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,issue_key,summary,issue_type,issue_status,issue_priority,description,comments,architectural_impact_manual,architectural_impact,TP,TN,FP,FN
0,CASSANDRA-11272,NullPointerException (NPE) during bootstrap st...,Bug,Resolved,Normal,After bootstrapping fails due to stream closed...,"['Which Cassandra version are you using? ', 'T...",YES,YES,1,0,0,0
1,CASSANDRA-1321,loadSchemaFromYaml should push migrations to c...,Improvement,Resolved,Normal,NaN,"['announces the last migration, which ends up ...",YES,YES,1,0,0,0
2,CASSANDRA-1641,auto-guessed memtable sizes are too high,Bug,Resolved,Normal,I've seen two cases now of the memtable sizes ...,"[""I'd like to introduce a dependency on number...",YES,YES,1,0,0,0
3,CASSANDRA-2296,"Scrub resulting in ""bloom filter claims to be ...",Bug,Resolved,Normal,Doing a scrub on a node which I upgraded from ...,"[""With debug logging turned on it looks like t...",YES,YES,1,0,0,0
4,CASSANDRA-3117,StorageServiceMBean is missing a getCompaction...,Bug,Resolved,Low,"Without a getter, you can assign a new value b...","['+1', 'committed.', 'Integrated in Cassandra-...",YES,YES,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,CASSANDRA-9631,Unnecessary required filtering for query on in...,Bug,Resolved,Normal,Let's create and populate a simple table compo...,['Is there a chance that CASSANDRA-8418 introd...,NO,NO,0,1,0,0
222,CASSANDRA-9636,Duplicate columns in selection causes Assertio...,Bug,Resolved,Normal,"Prior to CASSANDRA-9532, unaliased duplicate f...","[""I'm seeing the same error als on a query not...",NO,NO,0,1,0,0
223,CASSANDRA-9858,SelectStatement.Parameters fields should be in...,Bug,Resolved,Normal,SelectStatement.Parameters fields should be in...,['https://github.com/JeremiahDJordan/cassandra...,NO,NO,0,1,0,0
224,CASSANDRA-9880,ScrubTest.testScrubOutOfOrder should generate ...,Bug,Resolved,Urgent,ScrubTest#testScrubOutOfOrder is failing on tr...,"[""Patch attached as link.\nLet's see what cass...",NO,NO,0,1,0,0


In [5]:
colunas_uteis = ['issue_key', 'summary', 'architectural_impact_manual', 'architectural_impact']
df_im = df_inspection[colunas_uteis]
df_im_only_manual = df_im[df_im['architectural_impact_manual'] == 'YES']
df_im_only_manual_and_only_auto = df_im_only_manual[df_im_only_manual['architectural_impact']=='YES']
df_im_only_manual_and_only_auto

,issue_key,summary,architectural_impact_manual,architectural_impact
0,CASSANDRA-11272,NullPointerException (NPE) during bootstrap st...,YES,YES
1,CASSANDRA-1321,loadSchemaFromYaml should push migrations to c...,YES,YES
2,CASSANDRA-1641,auto-guessed memtable sizes are too high,YES,YES
3,CASSANDRA-2296,"Scrub resulting in ""bloom filter claims to be ...",YES,YES
4,CASSANDRA-3117,StorageServiceMBean is missing a getCompaction...,YES,YES
...,...,...,...,...
71,CASSANDRA-6689,Partially Off Heap Memtables,YES,YES
72,CASSANDRA-71,Range query support,YES,YES
73,CASSANDRA-7705,Safer Resource Management,YES,YES
74,CASSANDRA-8004,Run LCS for both repaired and unrepaired data,YES,YES


In [6]:
df_im_only_manual_and_only_auto.to_excel('yes_only_manual_and_only_auto.xlsx')

In [7]:
df_inspection_check = df_inspection[['issue_key', 'issue_type', 'summary', 'architectural_impact_manual','architectural_impact', 'TP', 'TN', 'FP', 'FN']]
df_inspection_check

,issue_key,issue_type,summary,architectural_impact_manual,architectural_impact,TP,TN,FP,FN
0,CASSANDRA-11272,Bug,NullPointerException (NPE) during bootstrap st...,YES,YES,1,0,0,0
1,CASSANDRA-1321,Improvement,loadSchemaFromYaml should push migrations to c...,YES,YES,1,0,0,0
2,CASSANDRA-1641,Bug,auto-guessed memtable sizes are too high,YES,YES,1,0,0,0
3,CASSANDRA-2296,Bug,"Scrub resulting in ""bloom filter claims to be ...",YES,YES,1,0,0,0
4,CASSANDRA-3117,Bug,StorageServiceMBean is missing a getCompaction...,YES,YES,1,0,0,0
...,...,...,...,...,...,...,...,...,...
221,CASSANDRA-9631,Bug,Unnecessary required filtering for query on in...,NO,NO,0,1,0,0
222,CASSANDRA-9636,Bug,Duplicate columns in selection causes Assertio...,NO,NO,0,1,0,0
223,CASSANDRA-9858,Bug,SelectStatement.Parameters fields should be in...,NO,NO,0,1,0,0
224,CASSANDRA-9880,Bug,ScrubTest.testScrubOutOfOrder should generate ...,NO,NO,0,1,0,0


In [8]:
true_positive = sum(df_inspection_check.TP)
true_negative = sum(df_inspection_check.TN)
false_positive = sum(df_inspection_check.FP)
false_negative = sum(df_inspection_check.FN)

print(f'Soma das métricas: {true_positive+true_negative+false_positive+false_negative}')
print(f'True Positive: {true_positive}, True Negative: {true_negative}, False Positive: {false_positive}, False Negative: {false_negative}')

Soma das métricas: 226
True Positive: 71, True Negative: 125, False Positive: 25, False Negative: 5


# Manual inspection

In [ ]:
df_inspection_check_manual = df_inspection_check[['issue_key', 'issue_type','summary','architectural_impact_manual']]
df_inspection_check_manual['ai'] = df_inspection_check_manual['architectural_impact_manual'].str.upper()
df_inspection_check_manual

<ipython-input-7-69427aaf3751>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inspection_check_manual['ai'] = df_inspection_check_manual['architectural_impact_manual'].str.upper()


,issue_key,issue_type,summary,architectural_impact_manual,ai
0,CASSANDRA-11272,Bug,NullPointerException (NPE) during bootstrap st...,YES,YES
1,CASSANDRA-1321,Improvement,loadSchemaFromYaml should push migrations to c...,YES,YES
2,CASSANDRA-1641,Bug,auto-guessed memtable sizes are too high,YES,YES
3,CASSANDRA-2296,Bug,"Scrub resulting in ""bloom filter claims to be ...",YES,YES
4,CASSANDRA-3117,Bug,StorageServiceMBean is missing a getCompaction...,YES,YES
...,...,...,...,...,...
221,CASSANDRA-9631,Bug,Unnecessary required filtering for query on in...,NO,NO
222,CASSANDRA-9636,Bug,Duplicate columns in selection causes Assertio...,NO,NO
223,CASSANDRA-9858,Bug,SelectStatement.Parameters fields should be in...,NO,NO
224,CASSANDRA-9880,Bug,ScrubTest.testScrubOutOfOrder should generate ...,NO,NO


In [ ]:
ai_manual = df_inspection_check_manual.ai.to_list()
conta_yes, conta_no = conta_yes_no(my_list=ai_manual)
print(f'Soma Yes and No: {conta_yes+conta_no}, Yes: {conta_yes}, No: {conta_no}')
print(f'%Yes: {round((conta_yes/(conta_yes+conta_no)*100),2)}, %No: {round((conta_no/(conta_yes+conta_no)*100),2)}')

Soma Yes and No: 226, Yes: 76, No: 150
%Yes: 33.63, %No: 66.37


# Semi-automated inspection using ChatGPT

In [ ]:
df_inspection_check_chagpt = df_inspection_check[['issue_key', 'issue_type','summary','architectural_impact']]
df_inspection_check_chagpt['ai'] = df_inspection_check_chagpt['architectural_impact']
df_inspection_check_chagpt['ai'] = df_inspection_check_chagpt['architectural_impact'].str.upper()
df_inspection_check_chagpt = df_inspection_check_chagpt[['issue_key', 'issue_type', 'summary', 'ai']]
df_inspection_check_chagpt

<ipython-input-9-a0ef252ef306>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inspection_check_chagpt['ai'] = df_inspection_check_chagpt['architectural_impact']
<ipython-input-9-a0ef252ef306>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inspection_check_chagpt['ai'] = df_inspection_check_chagpt['architectural_impact'].str.upper()


,issue_key,issue_type,summary,ai
0,CASSANDRA-11272,Bug,NullPointerException (NPE) during bootstrap st...,YES
1,CASSANDRA-1321,Improvement,loadSchemaFromYaml should push migrations to c...,YES
2,CASSANDRA-1641,Bug,auto-guessed memtable sizes are too high,YES
3,CASSANDRA-2296,Bug,"Scrub resulting in ""bloom filter claims to be ...",YES
4,CASSANDRA-3117,Bug,StorageServiceMBean is missing a getCompaction...,YES
...,...,...,...,...
221,CASSANDRA-9631,Bug,Unnecessary required filtering for query on in...,NO
222,CASSANDRA-9636,Bug,Duplicate columns in selection causes Assertio...,NO
223,CASSANDRA-9858,Bug,SelectStatement.Parameters fields should be in...,NO
224,CASSANDRA-9880,Bug,ScrubTest.testScrubOutOfOrder should generate ...,NO


In [ ]:
ai_chatgpt = df_inspection_check_chagpt.ai.to_list()
conta_yes, conta_no = conta_yes_no(my_list=ai_chatgpt)
print(f'Soma Yes and No: {conta_yes+conta_no}, Yes: {conta_yes}, No: {conta_no}')
print(f'%Yes: {round((conta_yes/(conta_yes+conta_no)*100),2)}, %No: {round((conta_no/(conta_yes+conta_no)*100),2)}')

Soma Yes and No: 226, Yes: 96, No: 130
%Yes: 42.48, %No: 57.52


# Cohens'Kappa


In [ ]:
# Converte a coluna architecture_issue para 0 (NO) ou 1 (YES)
dfa, dfb = convert_yes_no_to_int(dfa=df_inspection_check_manual, dfb=df_inspection_check_chagpt, my_column='ai')

<ipython-input-3-fb1b080fa0ff>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfa[my_column] = dfa[my_column].map(label_to_num)


In [ ]:
kappa_manual_chatgpt = calculate_cohens_kappa(dfa=df_inspection_check_manual, dfb=df_inspection_check_chagpt, my_column='ai')
print(round(kappa_manual_chatgpt, 3))

0.721


# Precision, recall, accuracy and F1-score LLM Model using ChatGTP by Prompt Engineering

In [ ]:
my_precision, my_recall, my_accuracy, my_f1_score = extra_metrics_yes(dfa=df_inspection_check_manual, dfb=df_inspection_check_chagpt, my_column='ai')
print(f'my_precision: {round(my_precision, 3)}, my_recall: {round(my_recall, 3)}, my_accuracy: {round(my_accuracy, 3)}, my_f1_score: {round(my_f1_score,3)}')

my_precision: 0.962, my_recall: 0.833, my_accuracy: 0.867, my_f1_score: 0.893


# Manual Inspection Training (extra)

In [10]:
colunas_uteis = ['issue_key', 'summary', 'architectural_impact_manual', 'architectural_impact']
df_im = df_inspection[colunas_uteis]
df_im_only_manual = df_im[df_im['architectural_impact_manual'] == 'YES']
df_im_only_manual_and_only_auto = df_im_only_manual[df_im_only_manual['architectural_impact']=='YES']

lista_issues_ia_yes = df_im_only_manual_and_only_auto['issue_key'].to_list()

In [13]:
!wget https://github.com/Technical-Debt-Large-Scale/my_validation/raw/main/cassandra/my_issues_to_inspection_cassandra.zip

--2023-12-22 13:11:49--  https://github.com/Technical-Debt-Large-Scale/my_validation/raw/main/cassandra/my_issues_to_inspection_cassandra.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Technical-Debt-Large-Scale/my_validation/main/cassandra/my_issues_to_inspection_cassandra.zip [following]
--2023-12-22 13:11:50--  https://raw.githubusercontent.com/Technical-Debt-Large-Scale/my_validation/main/cassandra/my_issues_to_inspection_cassandra.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 394310 (385K) [application/zip]
Saving to: ‘my_issues_to_inspection_cassandra.zip’

my_issues_to_inspec 1

In [ ]:
!wget https://github.com/Technical-Debt-Large-Scale/my_validation/raw/main/cassandra/my_results_inspection_cassandra.zip

In [14]:
!unzip my_issues_to_inspection_cassandra.zip

Archive:  my_issues_to_inspection_cassandra.zip
   creating: my_issues/
  inflating: my_issues/CASSANDRA-3338  
   creating: __MACOSX/
   creating: __MACOSX/my_issues/
  inflating: __MACOSX/my_issues/._CASSANDRA-3338  
  inflating: my_issues/CASSANDRA-708  
  inflating: __MACOSX/my_issues/._CASSANDRA-708  
  inflating: my_issues/CASSANDRA-3931  
  inflating: __MACOSX/my_issues/._CASSANDRA-3931  
  inflating: my_issues/CASSANDRA-4919  
  inflating: __MACOSX/my_issues/._CASSANDRA-4919  
  inflating: my_issues/CASSANDRA-12988  
  inflating: __MACOSX/my_issues/._CASSANDRA-12988  
  inflating: my_issues/CASSANDRA-6012  
  inflating: __MACOSX/my_issues/._CASSANDRA-6012  
  inflating: my_issues/CASSANDRA-15907  
  inflating: __MACOSX/my_issues/._CASSANDRA-15907  
  inflating: my_issues/CASSANDRA-1600  
  inflating: __MACOSX/my_issues/._CASSANDRA-1600  
  inflating: my_issues/CASSANDRA-8746  
  inflating: __MACOSX/my_issues/._CASSANDRA-8746  
  inflating: my_issues/CASSANDRA-2412  
  inflating

In [19]:
!unzip my_results_inspection_cassandra.zip

Archive:  my_results_inspection_cassandra.zip
   creating: merge_i1_i2/
  inflating: merge_i1_i2/CASSANDRA-4874.txt  
   creating: __MACOSX/merge_i1_i2/
  inflating: __MACOSX/merge_i1_i2/._CASSANDRA-4874.txt  
  inflating: merge_i1_i2/CASSANDRA-5571.txt  
  inflating: __MACOSX/merge_i1_i2/._CASSANDRA-5571.txt  
  inflating: merge_i1_i2/CASSANDRA-9411.txt  
  inflating: __MACOSX/merge_i1_i2/._CASSANDRA-9411.txt  
  inflating: merge_i1_i2/CASSANDRA-4647.txt  
  inflating: __MACOSX/merge_i1_i2/._CASSANDRA-4647.txt  
  inflating: merge_i1_i2/CASSANDRA-2975.txt  
  inflating: __MACOSX/merge_i1_i2/._CASSANDRA-2975.txt  
  inflating: merge_i1_i2/CASSANDRA-5015.txt  
  inflating: __MACOSX/merge_i1_i2/._CASSANDRA-5015.txt  
  inflating: merge_i1_i2/CASSANDRA-18029.txt  
  inflating: __MACOSX/merge_i1_i2/._CASSANDRA-18029.txt  
  inflating: merge_i1_i2/CASSANDRA-609.txt  
  inflating: __MACOSX/merge_i1_i2/._CASSANDRA-609.txt  
  inflating: merge_i1_i2/CASSANDRA-14411.txt  
  inflating: __MACOSX/

In [20]:
#!rm -rf issues_to_analysis

In [21]:
import os
import shutil

path_my_issues = '/content/my_issues'
path_my_results = '/content/merge_i1_i2'

for each in lista_issues_ia_yes:
  file_issue = each
  path_teste = '/'.join([path_my_issues, file_issue])
  file_issue_txt = file_issue + '.txt'
  path_teste_results = '/'.join([path_my_results, file_issue_txt])

  # Diretorio de destino
  new_issues_directory = 'issues_to_analysis'
  new_issues_results_directory = 'issue_results'

  if os.path.isdir(new_issues_directory) == False:
    os.mkdir(new_issues_directory)

  if os.path.isdir(new_issues_results_directory) == False:
    os.mkdir(new_issues_results_directory)

  # copia o arquivo da origem para o destino
  destination = '/'.join([new_issues_directory, file_issue])
  if os.path.exists(destination) == False:
    shutil.copyfile(src=path_teste, dst=destination)
    print(f'Arquivo {file_issue} copiado para {new_issues_directory}')

  # copia o arquivo da origem para o destino (resultado da inspecao chatgpt)
  destination_results = '/'.join([new_issues_results_directory, file_issue_txt])
  if os.path.exists(destination_results) == False:
    shutil.copyfile(src=path_teste_results, dst=destination_results)
    print(f'Arquivo {file_issue_txt} copiado para {new_issues_results_directory}')

Arquivo CASSANDRA-11272 copiado para issues_to_analysis
Arquivo CASSANDRA-11272.txt copiado para issue_results
Arquivo CASSANDRA-1321 copiado para issues_to_analysis
Arquivo CASSANDRA-1321.txt copiado para issue_results
Arquivo CASSANDRA-1641 copiado para issues_to_analysis
Arquivo CASSANDRA-1641.txt copiado para issue_results
Arquivo CASSANDRA-2296 copiado para issues_to_analysis
Arquivo CASSANDRA-2296.txt copiado para issue_results
Arquivo CASSANDRA-3117 copiado para issues_to_analysis
Arquivo CASSANDRA-3117.txt copiado para issue_results
Arquivo CASSANDRA-1035 copiado para issues_to_analysis
Arquivo CASSANDRA-1035.txt copiado para issue_results
Arquivo CASSANDRA-10580 copiado para issues_to_analysis
Arquivo CASSANDRA-10580.txt copiado para issue_results
Arquivo CASSANDRA-10866 copiado para issues_to_analysis
Arquivo CASSANDRA-10866.txt copiado para issue_results
Arquivo CASSANDRA-11944 copiado para issues_to_analysis
Arquivo CASSANDRA-11944.txt copiado para issue_results
Arquivo CAS

In [ ]:
!zip -r my_issues_to_analysis.zip issues_to_analysis

In [23]:
!zip -r my_issues_results.zip issue_results

  adding: issue_results/ (stored 0%)
  adding: issue_results/CASSANDRA-11272.txt (deflated 51%)
  adding: issue_results/CASSANDRA-8386.txt (deflated 53%)
  adding: issue_results/CASSANDRA-14204.txt (deflated 48%)
  adding: issue_results/CASSANDRA-12186.txt (deflated 54%)
  adding: issue_results/CASSANDRA-5918.txt (deflated 48%)
  adding: issue_results/CASSANDRA-1500.txt (deflated 48%)
  adding: issue_results/CASSANDRA-5426.txt (deflated 53%)
  adding: issue_results/CASSANDRA-10866.txt (deflated 50%)
  adding: issue_results/CASSANDRA-16545.txt (deflated 52%)
  adding: issue_results/CASSANDRA-15265.txt (deflated 56%)
  adding: issue_results/CASSANDRA-414.txt (deflated 45%)
  adding: issue_results/CASSANDRA-1321.txt (deflated 49%)
  adding: issue_results/CASSANDRA-1343.txt (deflated 47%)
  adding: issue_results/CASSANDRA-13272.txt (deflated 58%)
  adding: issue_results/CASSANDRA-15004.txt (deflated 50%)
  adding: issue_results/CASSANDRA-71.txt (deflated 53%)
  adding: issue_results/CASSAN